In [2]:
import sys
sys.path.append("../../../") # Where to find utilssphering
import utilssphering
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

In [230]:
# Read CSV
match_rep_df = pd.read_csv("../../../checkpoints/match_rep_df.csv")
match_rep_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,...,Size_MB_std,sphering,value_95_replicating,Percent_Replicating,channel_names,brightfield_z_plane_used,feature_channels_found,Percent_Matching,value_95_matching,cell_count
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,...,0.000144,True,0.191908,60.000000,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",23.255814,0.288099,2014937
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,...,0.000144,False,0.269727,53.333333,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",18.604651,0.320855,2014937
2,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,...,0.000183,True,0.269617,62.222222,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",18.604651,0.398249,2413350
3,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,...,0.000183,False,0.329074,45.555556,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",18.604651,0.389170,2413350
4,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,4,NaN,1,10,6,...,0.000142,True,0.205121,66.666667,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",23.255814,0.363114,2381443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO0117059_20X,Confocal,9,4.0,1,20,6,...,0.000583,False,0.628379,20.000000,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA",6.976744,0.772160,489099
358,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,...,0.000014,True,0.139090,56.666667,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA",18.604651,0.193171,452567
359,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,...,0.000014,False,0.666818,21.111111,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA",9.302326,0.790813,452567
360,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_40X_BRO0117059,JUMP-MOA_compound_platemap,BRO0117059_40x,Confocal,9,4.0,1,40,6,...,0.080201,True,0.193787,53.333333,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA",20.930233,0.234702,116595


In [222]:
def format_column_names(column_name):
    """
    For a given string, hyphens and underscores are replaced with spaces and 
    first letters are capitalised.
    """
    return column_name.title().replace("-", " ").replace("_", " ")

def aggregate_duplicates(df):
    """
    Group data on setting columns, calculate the mean for grouped rows
    (which are therefore duplicates).

    The returned dataframe will not contain Assay_Plate_Barcode, since 
    the unique barcodes cannot be reconciled into an aggregated mean, 
    so are therefore dropped.
    """
    # Columns to not be used for grouping
    non_group_cols = [
        "Assay_Plate_Barcode",
        "Batch",
        "Vendor",
        "value_95_replicating",
        "Percent_Replicating",
        "Size_MB",
        "Size_MB_std",
        "Percent_Matching",
        "value_95_matching",
        "cell_count",
        "Sites-SubSampled",
    ]
    # Find the columns that are not in non_group_cols
    diff = list(set(df.columns) - set(non_group_cols))
    # Group df by setting columns, find the mean, then reset the index
    df = df.groupby(diff, dropna=False, as_index=False).mean()
    return df

def make_leaderboard(df, columns, average_duplicates=True):
    """
    Process match_rep_df into a nice leaderboard
    """
    if average_duplicates:
        df = aggregate_duplicates(df)

    # Create an aggregation score for replicating/matching and normalize to the max value
    df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
    df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
    df = df.round({"Percent_Score": 1})

    # Sort based on the Percent_Score and then add Place (ie 1st, 2nd, 3rd etc.)
    df = df.sort_values("Percent_Score", ascending=False)[columns]
    df["Place"] = df.reset_index(drop=True).index+1
    
    # Move place column to first position
    df.insert(0, "Place", df.pop("Place"))

    # Format column_names
    df.rename(columns=format_column_names, inplace=True)
    df.rename(columns={"Z Plane": "Z Planes"}, inplace=True)
    df.rename(columns={"Aperture": "NA"}, inplace=True)
    df.rename(columns={"Dry Immersion": "Immersion"}, inplace=True)
    return df

# Microscopes used by each vendor

- Yokogawa US
  - Yokogawa CV8000
- Yokogawa Japan
  - CQ1
- Perkin Elmer
  - Opera Phenix Plus
- Molecular Devices
  - ImageXpress Micro Confocal (IXM-Confocal)
- Nikon
  - Eclipse Ti2 inverted microscope

In [195]:
moldev_cols = [
    # 'Percent_Score',
    # 'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    'spinning-disc',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    # 'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("MolDev")) &
    (~match_rep_df["Batch"].str.contains("siteSub")) &
    (match_rep_df["sphering"] == True)
].copy()

make_leaderboard(df, moldev_cols)

# df.to_csv("(1)-moldev_leaderboard.csv", index=False)

29
28


,Place,Modality,Binning,Magnification,NA,Number Of Channels,Z Planes,Spinning Disc,Sites
3,1,Confocal,1,20,0.75,6,1,no,9
2,2,Confocal,1,10,0.45,6,3,yes,4
0,3,Confocal,1,10,0.45,6,1,no,4
1,4,Confocal,1,10,0.45,6,1,yes,4
4,5,Confocal,1,20,0.75,6,1,yes,9
5,6,Confocal,1,20,0.75,6,3,yes,4


In [232]:
yokogawa_us_cols = [
    # 'Percent_Score',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    "simultaneous-excitation",
    'z_plane',
    "dry-immersion",
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Yokogawa_US")) &
    (~match_rep_df["Batch"].str.contains("siteSub")) &
    (match_rep_df["sphering"] == True)
].copy()

df["simultaneous-excitation"] = df["simultaneous-excitation"].astype("Int64")

make_leaderboard(df, yokogawa_us_cols)

# df.to_csv("(2)-yokogawa_US_leaderboard.csv", index=False)

,Place,Modality,Binning,Magnification,NA,Number Of Channels,Simultaneous Excitation,Z Planes,Immersion,Sites
2,1,Confocal,1,20,1.0,5,2,12,water,9
3,2,Confocal,1,20,1.0,6,4,12,water,9
0,3,Confocal,1,20,1.0,5,2,1,water,9
1,4,Confocal,1,10,0.4,6,2,12,dry,4
4,5,Confocal,1,20,1.0,6,4,12,water,9
5,6,Confocal,1,40,1.0,6,4,12,water,9


In [233]:
yokogawa_japan_cols = [
    # 'Percent_Score',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    "simultaneous-excitation",
    'z_plane',
    "dry-immersion",
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Yokogawa_Japan")) &
    (~match_rep_df["Batch"].str.contains("siteSub")) &
    (match_rep_df["sphering"] == True)
].copy()

make_leaderboard(df, yokogawa_japan_cols)

# df.to_csv("(3)-yokogawa_japan_leaderboard.csv", index=False)

,Place,Modality,Binning,Magnification,NA,Number Of Channels,Simultaneous Excitation,Z Planes,Immersion,Sites
0,1,Confocal,1,20,0.75,5,NaN,10,dry,9
1,2,Confocal,1,40,0.95,5,NaN,12,dry,2


In [220]:
nikon_cols = [
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    "dry-immersion",
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Nikon")) &
    (~match_rep_df["Batch"].str.contains("siteSub")) &
    (match_rep_df["sphering"] == True)
].copy()

make_leaderboard(df, nikon_cols)

# df.to_csv("(4)-nikon_leaderboard.csv", index=False)

29
26


,Place,Modality,Binning,Magnification,NA,Number Of Channels,Z Planes,Immersion,Sites
1,1,Widefield,1,20,0.75,4,1,dry,9
0,2,Widefield,1,10,0.45,4,1,dry,1


In [177]:
pe_cols = [
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    "dry-immersion",
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("PE")) &
    (~match_rep_df["Batch"].str.contains("siteSub")) &
    (match_rep_df["sphering"] == True)
].copy()

make_leaderboard(df, pe_cols)

# df.to_csv("(5)-pe_leaderboard.csv", index=False)

,Place,Modality,Binning,Magnification,NA,Number Of Channels,Z Planes,Immersion,Sites
3,1,Confocal,2,20,1.0,5,3,water,3
1,2,Confocal,2,20,1.0,5,1,water,3
5,3,Widefield,1,20,1.0,5,1,water,3
6,4,Widefield,2,20,1.0,5,1,water,3
9,5,Widefield,2,20,1.0,5,3,water,3
7,6,Widefield,1,20,1.0,5,3,water,3
8,7,Widefield,2,20,1.0,5,3,water,3
2,8,Confocal,1,20,1.0,5,3,water,3
0,9,Confocal,1,20,1.0,5,1,water,3
4,10,Confocal,2,20,1.0,5,3,water,3
